# Comparing Nonlinear Oscillators: Hopf, Stuart-Landau, and Van der Pol

This tutorial provides a comprehensive comparison of three fundamental nonlinear oscillators commonly used in computational neuroscience and dynamical systems theory:

1. **Hopf Oscillator** - The canonical normal form for oscillatory dynamics near a Hopf bifurcation
2. **Stuart-Landau Oscillator** - A simplified version of the Hopf normal form
3. **Van der Pol Oscillator** - A relaxation oscillator with nonlinear damping

We will explore:
- The mathematical formulation of each oscillator
- Subcritical (damped) vs. supercritical (sustained) regimes
- The qualitative difference between harmonic-like oscillations (Hopf/Stuart-Landau) and relaxation oscillations (Van der Pol)
- Phase portraits and time series visualizations

In [1]:
# Imports and Setup
import jax.numpy as jnp
import matplotlib.pyplot as plt

import brainstate
import brainunit as u
from brainmass import HopfStep, StuartLandauStep, VanDerPolStep

# Enable high-quality figure output
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 10

ImportError: cannot import name 'maybe_init_prefetch' from 'brainstate.nn._dynamics' (D:\codes\projects\brainstate\brainstate\nn\_dynamics.py)

## 1. Hopf Oscillator

The **Hopf oscillator** implements the supercritical Hopf normal form, which describes the generic behavior of a dynamical system near a Hopf bifurcation. In complex form:

$$\frac{dz}{dt} = (a + i\omega)z - \beta|z|^2 z$$

where $z = x + iy$. In real coordinates:

$$\dot{x} = (a - \beta r^2)x - \omega y$$
$$\dot{y} = (a - \beta r^2)y + \omega x$$

where $r^2 = x^2 + y^2$.

### Parameters:
- **a (bifurcation parameter)**: Controls the stability of the fixed point
  - $a < 0$: Stable fixed point at origin (subcritical/damped)
  - $a > 0$: Stable limit cycle (supercritical/sustained oscillations)
- **$\omega$ (angular frequency)**: Determines the oscillation frequency
- **$\beta$ (saturation coefficient)**: Controls the limit cycle amplitude ($\approx \sqrt{a/\beta}$ when $a > 0$)

## 2. Stuart-Landau Oscillator

The **Stuart-Landau oscillator** is the standard Hopf normal form with $\beta = 1$:

$$\dot{x} = (a - r^2)x - \omega y$$
$$\dot{y} = (a - r^2)y + \omega x$$

This is mathematically equivalent to the Hopf oscillator but with fixed saturation, making the limit cycle amplitude simply $\sqrt{a}$ when $a > 0$.

### Key Properties:
- Same bifurcation structure as Hopf
- Simpler parameterization (fewer degrees of freedom)
- Produces **nearly sinusoidal** (harmonic-like) oscillations

## 3. Van der Pol Oscillator

The **Van der Pol oscillator** is fundamentally different from the Hopf/Stuart-Landau oscillators. It arises from the second-order ODE:

$$\ddot{x} - \mu(1 - x^2)\dot{x} + x = 0$$

Using the Lienard transformation, this becomes:

$$\dot{x} = \mu\left(x - \frac{x^3}{3} - y\right)$$
$$\dot{y} = \frac{x}{\mu}$$

### Parameters:
- **$\mu$ (nonlinearity/damping parameter)**: Controls the oscillation character
  - $\mu \approx 0$: Nearly harmonic oscillations (like a linear oscillator)
  - $\mu \sim 1$: Nonlinear but smooth oscillations
  - $\mu \gg 1$: **Relaxation oscillations** with sharp "snapping" transitions

### Key Difference from Hopf/Stuart-Landau:
The Van der Pol oscillator has **nonlinear damping** that depends on the amplitude. Near the origin, the system is unstable (negative damping), while far from the origin, it is overdamped. This creates the characteristic relaxation oscillation pattern where the system slowly drifts along a "slow manifold" and then rapidly "snaps" to the opposite side.

In [ ]:
def simulate_oscillator(model, n_steps, dt, init_x=0.1, init_y=0.0):
    """
    Simulate an XY_Oscillator model and return time series.
    
    Parameters
    ----------
    model : XY_Oscillator
        An oscillator model (HopfStep, StuartLandauStep, or VanDerPolStep)
    n_steps : int
        Number of simulation steps
    dt : float
        Time step in milliseconds
    init_x : float
        Initial x value (default: 0.1)
    init_y : float
        Initial y value (default: 0.0)
    
    Returns
    -------
    t : jnp.ndarray
        Time array
    x_hist : jnp.ndarray
        x trajectory
    y_hist : jnp.ndarray
        y trajectory
    """
    with brainstate.environ.context(dt=dt * u.ms):
        model.init_state()

        # Set initial conditions
        model.x.value = jnp.array([init_x])
        model.y.value = jnp.array([init_y])

        x_hist, y_hist = brainstate.transform.for_loop(lambda i: model.update(), jnp.arange(n_steps))
        t = jnp.arange(n_steps) * dt
        return t, jnp.array(x_hist), jnp.array(y_hist)

## Simulation Parameters

We will simulate each oscillator in two regimes:

### Subcritical (Stable Fixed Point):
- **Hopf/Stuart-Landau**: $a = -0.5$ (origin is stable, oscillations decay)
- **Van der Pol**: $\mu = 0.1$ (weakly nonlinear, nearly harmonic decay)

### Supercritical (Stable Limit Cycle):
- **Hopf/Stuart-Landau**: $a = 0.5$ (sustained oscillations)
- **Van der Pol**: $\mu = 1.0$ (standard limit cycle)

### Relaxation Oscillations:
- **Van der Pol**: $\mu = 5.0$ (high nonlinearity, sharp transitions)

In [ ]:
# Simulation settings
dt = 0.1  # Time step (ms)
n_steps = 3000  # Number of steps
init_x, init_y = 0.5, 0.0  # Initial conditions

# --- Hopf Oscillator ---
# Subcritical (damped)
hopf_sub = HopfStep(in_size=1, a=-0.5, w=0.2, beta=1.0, method='rk4')
t_hopf_sub, x_hopf_sub, y_hopf_sub = simulate_oscillator(
    hopf_sub, n_steps, dt, init_x, init_y
)

# Supercritical (limit cycle)
hopf_sup = HopfStep(in_size=1, a=0.5, w=0.2, beta=1.0, method='rk4')
t_hopf_sup, x_hopf_sup, y_hopf_sup = simulate_oscillator(
    hopf_sup, n_steps, dt, init_x, init_y
)

In [2]:
# --- Stuart-Landau Oscillator ---
# Subcritical (damped)
sl_sub = StuartLandauStep(in_size=1, a=-0.5, w=0.2, method='rk4')
t_sl_sub, x_sl_sub, y_sl_sub = simulate_oscillator(
    sl_sub, n_steps, dt, init_x, init_y
)

# Supercritical (limit cycle)
sl_sup = StuartLandauStep(in_size=1, a=0.5, w=0.2, method='rk4')
t_sl_sup, x_sl_sup, y_sl_sup = simulate_oscillator(
    sl_sup, n_steps, dt, init_x, init_y
)

NameError: name 'StuartLandauStep' is not defined

In [ ]:
# --- Van der Pol Oscillator ---
# Standard limit cycle (mu=1)
vdp_std = VanDerPolStep(in_size=1, mu=1.0, method='rk4')
t_vdp_std, x_vdp_std, y_vdp_std = simulate_oscillator(
    vdp_std, n_steps, dt, init_x, init_y
)

# Relaxation oscillations (high mu)
vdp_relax = VanDerPolStep(in_size=1, mu=5.0, method='rk4')
t_vdp_relax, x_vdp_relax, y_vdp_relax = simulate_oscillator(
    vdp_relax, n_steps, dt, init_x, init_y
)

## Main Visualization: 3x2 Subplot Grid

- **Top Row**: Time series ($x$ vs. $t$)
- **Bottom Row**: Phase portraits ($y$ vs. $x$)
- **Columns**: Hopf | Stuart-Landau | Van der Pol

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(12, 8))

# Colors
color_sub = '#2ecc71'    # Green for subcritical/damped
color_sup = '#e74c3c'    # Red for supercritical/limit cycle
color_relax = '#9b59b6'  # Purple for relaxation

# === Column 1: Hopf Oscillator ===
# Time series
ax = axes[0, 0]
ax.plot(t_hopf_sub, x_hopf_sub, color=color_sub, linestyle='--', 
        label=f'a = -0.5 (damped)', alpha=0.8)
ax.plot(t_hopf_sup, x_hopf_sup, color=color_sup, 
        label=f'a = 0.5 (limit cycle)', alpha=0.8)
ax.set_xlabel('Time (ms)')
ax.set_ylabel('x(t)')
ax.set_title('Hopf Oscillator')
ax.legend(loc='upper right', fontsize=8)
ax.set_xlim([0, t_hopf_sub[-1]])

# Phase portrait
ax = axes[1, 0]
ax.plot(x_hopf_sub, y_hopf_sub, color=color_sub, linestyle='--', 
        label='a = -0.5', alpha=0.8)
ax.plot(x_hopf_sup, y_hopf_sup, color=color_sup, 
        label='a = 0.5', alpha=0.8)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('Phase Portrait')
ax.legend(loc='upper right', fontsize=8)
ax.set_aspect('equal', adjustable='box')

# === Column 2: Stuart-Landau Oscillator ===
# Time series
ax = axes[0, 1]
ax.plot(t_sl_sub, x_sl_sub, color=color_sub, linestyle='--', 
        label=f'a = -0.5 (damped)', alpha=0.8)
ax.plot(t_sl_sup, x_sl_sup, color=color_sup, 
        label=f'a = 0.5 (limit cycle)', alpha=0.8)
ax.set_xlabel('Time (ms)')
ax.set_ylabel('x(t)')
ax.set_title('Stuart-Landau Oscillator')
ax.legend(loc='upper right', fontsize=8)
ax.set_xlim([0, t_sl_sub[-1]])

# Phase portrait
ax = axes[1, 1]
ax.plot(x_sl_sub, y_sl_sub, color=color_sub, linestyle='--', 
        label='a = -0.5', alpha=0.8)
ax.plot(x_sl_sup, y_sl_sup, color=color_sup, 
        label='a = 0.5', alpha=0.8)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('Phase Portrait')
ax.legend(loc='upper right', fontsize=8)
ax.set_aspect('equal', adjustable='box')

# === Column 3: Van der Pol Oscillator ===
# Time series
ax = axes[0, 2]
ax.plot(t_vdp_std, x_vdp_std, color=color_sup, 
        label=r'$\mu$ = 1.0 (standard)', alpha=0.8)
ax.plot(t_vdp_relax, x_vdp_relax, color=color_relax, 
        label=r'$\mu$ = 5.0 (relaxation)', alpha=0.8)
ax.set_xlabel('Time (ms)')
ax.set_ylabel('x(t)')
ax.set_title('Van der Pol Oscillator')
ax.legend(loc='upper right', fontsize=8)
ax.set_xlim([0, t_vdp_std[-1]])

# Phase portrait
ax = axes[1, 2]
ax.plot(x_vdp_std, y_vdp_std, color=color_sup, 
        label=r'$\mu$ = 1.0', alpha=0.8)
ax.plot(x_vdp_relax, y_vdp_relax, color=color_relax, 
        label=r'$\mu$ = 5.0', alpha=0.8)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('Phase Portrait')
ax.legend(loc='upper right', fontsize=8)

plt.tight_layout()
plt.show()

## Key Observations

### Hopf and Stuart-Landau Oscillators:
- **Harmonic-like motion**: The oscillations are nearly sinusoidal
- **Circular phase portraits**: The limit cycles form nearly perfect circles in the (x, y) plane
- **Amplitude control via bifurcation parameter**: The limit cycle amplitude is $\sqrt{a/\beta}$ for Hopf and $\sqrt{a}$ for Stuart-Landau
- **Smooth transients**: When starting away from the limit cycle, trajectories smoothly spiral in/out

### Van der Pol Oscillator:
- **Nonlinear "snapping" behavior**: At high $\mu$, the oscillator exhibits **relaxation oscillations**
- **Distorted phase portraits**: The limit cycle is far from circular, showing the characteristic "peanut" shape
- **Two timescales**: Slow drift along the "nullcline" followed by rapid jumps
- **Sharper waveforms**: The time series shows flat plateaus with sharp transitions

### Why the Difference?
The fundamental distinction lies in the source of nonlinearity:
- **Hopf/Stuart-Landau**: Nonlinearity is in the *amplitude-dependent growth rate* ($-r^2$), which saturates the amplitude but preserves the sinusoidal shape
- **Van der Pol**: Nonlinearity is in the *damping term* ($\mu(1-x^2)$), which creates amplitude-dependent dissipation, leading to relaxation dynamics

In [ ]:
# Detailed comparison: Van der Pol at different mu values
fig, axes = plt.subplots(2, 3, figsize=(12, 6))

mu_values = [0.5, 2.0, 8.0]
colors = ['#3498db', '#e67e22', '#c0392b']

for i, (mu, color) in enumerate(zip(mu_values, colors)):
    vdp = VanDerPolStep(in_size=1, mu=mu, method='rk4')
    t, x, y = simulate_oscillator(vdp, 2000, 0.1, 0.5, 0.0)
    
    # Time series
    axes[0, i].plot(t, x, color=color, linewidth=1.5)
    axes[0, i].set_xlabel('Time (ms)')
    axes[0, i].set_ylabel('x(t)')
    axes[0, i].set_title(f'$\\mu$ = {mu}')
    axes[0, i].set_xlim([0, 200])
    
    # Phase portrait
    axes[1, i].plot(x, y, color=color, linewidth=1.5)
    axes[1, i].set_xlabel('x')
    axes[1, i].set_ylabel('y')
    axes[1, i].set_title('Phase Portrait')

plt.suptitle('Van der Pol: Transition to Relaxation Oscillations', fontsize=12, y=1.02)
plt.tight_layout()
plt.show()

## Summary Table

| Property | Hopf Oscillator | Stuart-Landau | Van der Pol |
|----------|-----------------|---------------|-------------|
| Bifurcation parameter | $a$ | $a$ | Always oscillates for $\mu > 0$ |
| Limit cycle amplitude | $\sqrt{a/\beta}$ | $\sqrt{a}$ | $\approx 2$ (fixed) |
| Waveform shape | Sinusoidal | Sinusoidal | Relaxation (at high $\mu$) |
| Phase portrait | Circular | Circular | Distorted |
| Nonlinearity source | Amplitude saturation | Amplitude saturation | Nonlinear damping |
| Neuroscience use | Neural oscillators | Phase amplitude coupling | Bursting neurons |

In [ ]:
# Interactive exploration: Sweep bifurcation parameter 'a' for Stuart-Landau
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

a_values = [-0.5, -0.2, 0.0, 0.2, 0.5, 1.0]
cmap = plt.cm.coolwarm

for i, a_val in enumerate(a_values):
    color = cmap(i / (len(a_values) - 1))
    sl = StuartLandauStep(in_size=1, a=a_val, w=0.3, method='rk4')
    t, x, y = simulate_oscillator(sl, 2000, 0.1, 0.5, 0.1)
    
    # Use only steady-state portion
    steady_idx = len(t) // 2
    
    axes[0].plot(t[:500], x[:500], color=color, label=f'a = {a_val}', alpha=0.8)
    axes[1].plot(x[steady_idx:], y[steady_idx:], color=color, label=f'a = {a_val}', alpha=0.8)

axes[0].set_xlabel('Time (ms)')
axes[0].set_ylabel('x(t)')
axes[0].set_title('Stuart-Landau: Bifurcation Sweep')
axes[0].legend(fontsize=8, loc='upper right')

axes[1].set_xlabel('x')
axes[1].set_ylabel('y')
axes[1].set_title('Phase Portraits (Steady State)')
axes[1].set_aspect('equal', adjustable='box')
axes[1].legend(fontsize=8, loc='upper right')

plt.tight_layout()
plt.show()

## Conclusion

This tutorial demonstrated the fundamental differences between three important nonlinear oscillators:

1. **Hopf and Stuart-Landau oscillators** exhibit smooth, harmonic-like oscillations controlled by a bifurcation parameter. They are widely used to model neural population oscillations where the amplitude needs to be tunable.

2. **Van der Pol oscillator** shows qualitatively different behavior with its relaxation oscillations at high $\mu$. This makes it suitable for modeling bursting neurons and other systems with distinct "on" and "off" phases.

The brainmass library provides efficient implementations of all three oscillators through the `HopfStep`, `StuartLandauStep`, and `VanDerPolStep` classes, all inheriting from the common `XY_Oscillator` base class with flexible integration methods.